RUN THIS IF YOU DON'T HAVE AN ENVIROMENT WITH THE REQUISITES INSTALLED

In [ ]:
# %pip install qutip
%pip install scipy --upgrade
%pip install numpy --upgrade
%pip install matplotlib --upgrade

RUN THIS BEFORE RUNNING THE OTHER CELLS

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# import qutip as qt
import scipy as sp
from scipy.integrate import odeint

# Model of S. Magaletti

This is a implementation of the model of [Magaletti et al.(2020) "Modelling Rabi oscillations for widefield radiofrequency imaging in nitrogen-vacancy centers in diamond"](https://arxiv.org/abs/2309.06203) using Scipy ODEint. All constants are given in the paper linked above.

In [ ]:
# Constants (MHz)
omr = 15.7
Wp = 1.9
omega = 12
gamma_2 = 0.5
k41 = 62.7
k52 = 62.7
k63 = 62.7
k57 = 80.0
k67 = 80.0
k47 = 12.97
k71 = 3.45
k72 = 1.08
k73 = 1.08
# Initial conditions
n1, n2, n3, n4 = 1 / 3, 1 / 3, 1 / 3, 0.0
n5, n6, n7, nc = 0.0, 0.0, 0.0, 0.0
n0 = [n1, n2, n3, n4, n5, n6, n7, nc]


def dns(n, t, W_p, om_r):
    """Returns the time derivative of the density matrix elements based on whether the laser and MW are on or off
    Returns: dn1, dn2, dn3, dn4, dn5, dn6, dn7, dnc (list) - list of the time derivatives of the density matrix elements
    """
    dn1 = -1 * W_p * n[0] + k41 * n[3] + k71 * n[6] + om_r * n[7]
    dn2 = -1 * W_p * n[1] + k52 * n[4] + k72 * n[6] - om_r * n[7]
    dn3 = -1 * W_p * n[2] + k63 * n[5] + k73 * n[6]
    dn4 = W_p * n[0] - k41 * n[3] - k47 * n[3]
    dn5 = W_p * n[1] - k52 * n[4] - k57 * n[4]
    dn6 = W_p * n[2] - k63 * n[5] - k67 * n[5]
    dn7 = k67 * n[5] + k57 * n[4] + k47 * n[3] - (k71 + k72 + k73) * n[6]
    dnc = -gamma_2 * n[7] + om_r * (n[1] - n[0]) / 2
    return dn1, dn2, dn3, dn4, dn5, dn6, dn7, dnc


# colors for each level
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "forestgreen",
]
t = np.linspace(0, 10, 5000)
# Evolution and plot with laser
sol = odeint(dns, n0, t, args=(Wp, 0.0))
plt.figure(figsize=(10, 4))
for k in range(8):
    plt.plot(t, sol[:, k], color=colors[k])

# Free evolution and plot
n1 = sol[-1, :]
t1 = t[-1] + np.linspace(0, 1, 1000)
sol = odeint(dns, n1, t1, args=(0.0, 0.0))
for k in range(7):
    plt.plot(t1, sol[:, k], label=f"$n_{k+1}$", color=colors[k])
plt.plot(t1, sol[:, -1], label=f"$n_c$", color=colors[-1])

# Evolution and plot with microwave
n2 = sol[-1, :]
t2 = t1[-1] + np.linspace(0, 1, 1000)
sol = odeint(dns, n2, t2, args=(0.0, omr))
for k in range(8):
    plt.plot(t2, sol[:, k], color=colors[k])

# Fill for the different regimes
plt.fill_betweenx(
    [-0.4, 0.85], t[0], t[-1], color="palegreen", alpha=0.7, label=f"Laser ON"
)
plt.fill_betweenx(
    [-0.4, 0.85], t1[0], t1[-1], color="paleturquoise", alpha=0.7, label=f"MS depl"
)
plt.fill_betweenx(
    [-0.4, 0.85], t2[0], t2[-1], color="orchid", alpha=0.7, label=f"MW ON"
)

plt.ylabel("Population")
plt.xlabel("Time($\mu$s)")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")